In [1]:
import pandas as pd
from neo4j import GraphDatabase
import json
import random
import requests
import base64

from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

# Некоторое описание исходных данных

In [2]:
df = pd.read_csv('data_test.csv', delimiter=';')
df.rename(columns = dict(zip(df.columns, ['id', 'Person_1', 'Person_2'])), inplace=True)
df.head()

,id,Person_1,Person_2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


In [3]:
#проверка на наличие дубликатов
df.duplicated().sum()

0

In [4]:
#Проверка на наличие пропусков
df.isna().sum()

id          0
Person_1    0
Person_2    0
dtype: int64

In [6]:
# Количественные характеристики будущей базы данных

print(f'Количествов отношений = {df.shape[0]}')
fio_list = set(df.Person_1.to_list()+ df.Person_2.to_list()) # Множество ФИО. Будет использоваться в запросе далее.
print(f'Количество узлов = {len(fio_list)}')

Количествов отношений = 5000
Количество узлов = 9899


# Работа с neo4j

In [5]:
uri, username, password, db = 'neo4j://localhost:7687', 'neo4j', '********', 'neo4j'

In [11]:
#Заполнение базы данных

def create_event(tx, event): #функция транзакции создания события
    tx.run("""MERGE (p1:Person {name: $event.Person_1})
              MERGE (p2:Person {name: $event.Person_2})
              MERGE (p1)-[:RELATION {id: $event.id}]-(p2)
           """, event = event
    )

with GraphDatabase.driver(uri, auth = (username, password)) as driver:
    with driver.session(database=db) as session:

        for row in range(len(df)):
            event = df.iloc[row].to_dict()
            session.execute_write(create_event, event)

In [144]:
# Запросы к базе данных
random.seed(45)
fio = random.choice(list(fio_list))
print(f'ФИО: {fio}')

d = {"statements": [{"statement": "MATCH ans = ({name: $props.name})--() RETURN ans", 
                     "parameters": {"props": {"name": fio}},
                    }]}

url = "http://localhost:7474/db/neo4j/tx/commit"

headers = {'Accept': 'application/json',
           'Content-Type': 'application/json',
           'Authorization': 'Basic ' + base64.b64encode(f'{username}:{password}'.encode('utf-8')).decode()
           }

r =requests.post(url, data=json.dumps(d), headers=headers)

result = []
for event in json.loads(r.text)['results'][0]['data'][0]['row']:
    _a = dict(zip(['person_1', 'id', 'person_2'], (list(k.values())[0] for k in event)))
    result.append(json.dumps(_a))
result

ФИО: Обелова Кристина Ильдаровна

Формат словаря
{'person_1': 'Обелова Кристина Ильдаровна', 'id': 791076, 'person_2': 'Божок Виталий Яковлев'}

Формат JSON


['{"person_1": "\\u041e\\u0431\\u0435\\u043b\\u043e\\u0432\\u0430 \\u041a\\u0440\\u0438\\u0441\\u0442\\u0438\\u043d\\u0430 \\u0418\\u043b\\u044c\\u0434\\u0430\\u0440\\u043e\\u0432\\u043d\\u0430", "id": 791076, "person_2": "\\u0411\\u043e\\u0436\\u043e\\u043a \\u0412\\u0438\\u0442\\u0430\\u043b\\u0438\\u0439 \\u042f\\u043a\\u043e\\u0432\\u043b\\u0435\\u0432"}']

# Работа с NebulaGraph

In [ ]:
config = Config()
config.max_connection_pool_size = 10
keys_list = {j:f'p_{i}' for i,j in enumerate(fio_list,1)} #id номера персоналей для формирования vid

In [138]:
#Загрузка данных в БД
connection_pool = ConnectionPool()
ok = connection_pool.init([('127.0.0.1', 9669)], config)

with connection_pool.session_context('root', '********') as session:
    
    session.execute('''CREATE SPACE IF NOT EXISTS task_7(partition_num=15, replica_factor=1, vid_type=FIXED_STRING(32));
                       USE task_7;
                       CREATE TAG IF NOT EXISTS  Person(name string);
                       CREATE EDGE IF NOT EXISTS event(id int);''')
    
    for row in range(len(df)):
        p1, p2, w = df.loc[row, 'Person_1'], df.loc[row,'Person_2'], df.loc[row, 'id']
        p1_vid, p2_vid = keys_list[p1], keys_list[p2]
        session.execute(f'''INSERT VERTEX IF NOT EXISTS Person(name) VALUES "{p1_vid}":("{p1}"), "{p2_vid}":("{p2}");
                            INSERT EDGE IF NOT EXISTS event(id) VALUES "{p1_vid}"->"{p2_vid}":({w});''')

connection_pool.close()

In [206]:
#Запрос данных из БД
connection_pool = ConnectionPool()
ok = connection_pool.init([('127.0.0.1', 9669)], config)

with connection_pool.session_context('root', '96ZRyX36') as session:
    
    session.execute('''USE task_7;''')
    r = session.execute('''LOOKUP ON Person
                             WHERE Person.name=="Медведева Дарья Алексеевна" 
                             YIELD id(vertex) AS p | GO FROM $-.p OVER event BIDIRECT
                             YIELD properties($$).name AS person_1, properties(edge).id AS id, properties($^).name AS person_2''')
    
connection_pool.close()

result = []
for row in range(r.row_size()):
    _dict = dict.fromkeys(r.keys(), None)
    for k in r.keys():
        if k in ['id']:
            _dict[k] = r.column_values(k)[row].as_int()
        else:
            _dict[k] = r.column_values(k)[row].as_string()
    result.append(json.dumps(_dict))

result

['{"person_1": "\\u041a\\u043e\\u043d\\u0434\\u0440\\u0430\\u0442\\u044c\\u0435\\u0432 \\u0411\\u043e\\u0440\\u0438\\u0441 \\u0413\\u0435\\u0440\\u043c\\u0430\\u043d\\u043e\\u0432\\u0438\\u0447", "id": 87253, "person_2": "\\u041c\\u0435\\u0434\\u0432\\u0435\\u0434\\u0435\\u0432\\u0430 \\u0414\\u0430\\u0440\\u044c\\u044f \\u0410\\u043b\\u0435\\u043a\\u0441\\u0435\\u0435\\u0432\\u043d\\u0430"}',
 '{"person_1": "\\u041f\\u043e\\u043c\\u044b\\u043a\\u0430\\u043b\\u043e\\u0432\\u0430 \\u0422\\u0430\\u043c\\u0430\\u0440\\u0430 \\u0424\\u0435\\u0434\\u043e\\u0440\\u043e\\u0432\\u043d\\u0430", "id": 196243, "person_2": "\\u041c\\u0435\\u0434\\u0432\\u0435\\u0434\\u0435\\u0432\\u0430 \\u0414\\u0430\\u0440\\u044c\\u044f \\u0410\\u043b\\u0435\\u043a\\u0441\\u0435\\u0435\\u0432\\u043d\\u0430"}',
 '{"person_1": "\\u041f\\u0447\\u0435\\u043b\\u0438\\u043d\\u0446\\u0435\\u0432 \\u0410\\u0440\\u0442\\u0443\\u0440 \\u0413\\u043b\\u0435\\u0431\\u043e\\u0432\\u0438\\u0447", "id": 580478, "person_2": "\\u